In [5]:
import pandas as pd

# Try loading with 'latin-1' encoding
pastoral_df = pd.read_csv('pastoral.csv', encoding='latin-1')

# Display basic information about the dataset
print("Dataset Shape:", pastoral_df.shape)
print("\
First few rows:")
print(pastoral_df.head())
print("\
Columns:", pastoral_df.columns.tolist())

Dataset Shape: (144, 3)
First few rows:
   code                                    job description       label
0  1398                   Managing a caseload of graduates  Response B
1  1398  Providing support, advice, and information to ...  Response B
2  1398     Tracking and reporting on graduate progression  Response B
3  1398  Engaging with students regarding their well-be...  Response B
4  1368  Providing leadership and ensuring compliance w...  Response B
Columns: ['code', 'job description', 'label']


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')

# Prepare features (X) and target (y)
X = pastoral_df['job description']
y = pastoral_df['label']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transform text data to TF-IDF features
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = rf_model.predict(X_test_tfidf)

# Print classification report
print("Model Performance:")
print(classification_report(y_test, y_pred))

# Save the model and vectorizer
import joblib
joblib.dump(rf_model, 'pastoral_care_model.joblib')
joblib.dump(tfidf, 'tfidf_vectorizer.joblib')
print("\
Model and vectorizer saved successfully!")

Model Performance:
              precision    recall  f1-score   support

  Response A       0.67      0.36      0.47        11
  Response B       0.70      0.89      0.78        18

    accuracy                           0.69        29
   macro avg       0.68      0.63      0.63        29
weighted avg       0.68      0.69      0.66        29

Model and vectorizer saved successfully!


In [7]:
import joblib

def predict_pastoral_care(job_description):
    # Load the saved model and vectorizer
    model = joblib.load('pastoral_care_model.joblib')
    vectorizer = joblib.load('tfidf_vectorizer.joblib')
    
    # Transform the input text
    job_description_tfidf = vectorizer.transform([job_description])
    
    # Make prediction
    prediction = model.predict(job_description_tfidf)[0]
    
    # Get probability scores
    probabilities = model.predict_proba(job_description_tfidf)[0]
    confidence = max(probabilities) * 100
    
    return prediction, confidence

# Test the function with a sample job description
test_description = "Providing mentoring and support to students, including regular one-on-one sessions and workshops on personal development. Responsible for student welfare and maintaining records of support provided."

prediction, confidence = predict_pastoral_care(test_description)
print("Job Description:", test_description)
print("\
Predicted Label:", prediction)
print("Confidence: {:.2f}%".format(confidence))

Job Description: Providing mentoring and support to students, including regular one-on-one sessions and workshops on personal development. Responsible for student welfare and maintaining records of support provided.
Predicted Label: Response A
Confidence: 65.00%


In [8]:
# Let's try a different approach using BERT with transformers
%pip install transformers==4.30.2 torch==2.0.1

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd

# Load the CSV file
pastoral_df = pd.read_csv('pastoral.csv')

# Display the first few rows of the dataframe to understand its structure
print(pastoral_df.head())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 10477: invalid continuation byte

In [11]:
%pip install nlpaug==1.1.10

# Re-import the necessary modules after installation
import nlpaug.augmenter.word as naw
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

# Initialize augmenter
aug = naw.SynonymAug(aug_p=0.3)

# Function to augment text while preserving key terms
def augment_text(text):
    sentences = sent_tokenize(text)
    augmented_sentences = []
    for sentence in sentences:
        try:
            aug_text = aug.augment(sentence)[0]
            augmented_sentences.append(aug_text)
        except:
            augmented_sentences.append(sentence)
    return ' '.join(augmented_sentences)

# Augment the dataset
augmented_data = []
for _, row in df.iterrows():
    # Original data
    augmented_data.append({
        'code': row['code'],
        'job description': row['job description'],
        'label': row['label']
    })
    
    # Augmented data
    for _ in range(2):  # Create 2 augmented versions
        augmented_data.append({
            'code': row['code'],
            'job description': augment_text(row['job description']),
            'label': row['label']
        })

# Create new dataframe with augmented data
augmented_df = pd.DataFrame(augmented_data)

print("Original dataset size:", len(df))
print("Augmented dataset size:", len(augmented_df))
print("\
Sample of augmented data:")
print(augmented_df.head(3))

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leandrenash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NameError: name 'df' is not defined

In [12]:
# Load the CSV file and verify its structure
import pandas as pd

df = pd.read_csv('pastoral.csv', encoding='latin-1')
print("DataFrame Shape:", df.shape)
print("\
Columns:", df.columns.tolist())
print("\
First few rows:")
print(df.head())

DataFrame Shape: (144, 3)
Columns: ['code', 'job description', 'label']
First few rows:
   code                                    job description       label
0  1398                   Managing a caseload of graduates  Response B
1  1398  Providing support, advice, and information to ...  Response B
2  1398     Tracking and reporting on graduate progression  Response B
3  1398  Engaging with students regarding their well-be...  Response B
4  1368  Providing leadership and ensuring compliance w...  Response B


In [13]:
# Prepare the data for training
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Create separate labels for each question (40, 41, 42)
df['q40_label'] = df['label']
df['q41_label'] = df['label']
df['q42_label'] = df['label']

# Initialize label encoder
le = LabelEncoder()
df['q40_label_encoded'] = le.fit_transform(df['q40_label'])
df['q41_label_encoded'] = le.transform(df['q41_label'])
df['q42_label_encoded'] = le.transform(df['q42_label'])

# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Custom dataset class
class JobDescriptionDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = tokenizer(text, return_tensors='pt', padding='max_length', 
                           truncation=True, max_length=512)
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create datasets
train_dataset = JobDescriptionDataset(
    train_df['job description'].values,
    train_df[['q40_label_encoded', 'q41_label_encoded', 'q42_label_encoded']].values
)

test_dataset = JobDescriptionDataset(
    test_df['job description'].values,
    test_df[['q40_label_encoded', 'q41_label_encoded', 'q42_label_encoded']].values
)

print("Training samples:", len(train_dataset))
print("Testing samples:", len(test_dataset))
print("\
Label classes:", le.classes_)

# Define the model architecture
class JobClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(0.3)  # Increased dropout for better regularization
        self.classifier = nn.Linear(768, n_classes * 3)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits.view(-1, 3, 2)

# Initialize model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = JobClassifier(n_classes=2).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)  # Lower learning rate
criterion = nn.CrossEntropyLoss()

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

print(f"Training on {device}")

# Training loop with early stopping
n_epochs = 10
best_loss = float('inf')
patience = 3
no_improve = 0

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{n_epochs}'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        
        loss = 0
        for i in range(3):
            loss += criterion(outputs[:, i], labels[:, i])
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1} - Average loss: {avg_loss:.4f}')
    
    # Early stopping check
    if avg_loss < best_loss:
        best_loss = avg_loss
        no_improve = 0
    else:
        no_improve += 1
        
    if no_improve >= patience:
        print("Early stopping triggered!")
        break

/Users/leandrenash/anaconda3/envs/myvirutalenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/leandrenash/anaconda3/envs/myvirutalenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training samples: 115
Testing samples: 29
Label classes: ['Response A' 'Response B']


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training on cpu


Epoch 1/10: 100%|██████████| 15/15 [00:50<00:00,  3.34s/it]


...

Epoch 8 - Average loss: 0.9074


Epoch 9/10: 100%|██████████| 15/15 [00:53<00:00,  3.55s/it]


Epoch 9 - Average loss: 0.7095


Epoch 10/10: 100%|██████████| 15/15 [00:46<00:00,  3.08s/it]

Epoch 10 - Average loss: 0.3736


In [14]:
# Evaluation function
def evaluate_model(model, data_loader):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask)
            
            for i in range(3):  # For each question
                _, preds = torch.max(outputs[:, i], dim=1)
                correct_predictions += torch.sum(preds == labels[:, i])
                total_predictions += len(labels)
    
    accuracy = correct_predictions.double() / total_predictions
    return accuracy.item()

# Evaluate the model
accuracy = evaluate_model(model, test_loader)
print(f'Model Accuracy on Test Set: {accuracy * 100:.2f}%')

# Function to predict labels for a new job description
def predict_labels(job_description):
    model.eval()
    encoding = tokenizer(
        job_description,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=512
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predictions = []
        probabilities = []
        
        for i in range(3):  # For each question
            probs = torch.softmax(outputs[:, i], dim=1)
            confidence, preds = torch.max(probs, dim=1)
            predictions.append(le.inverse_transform(preds.cpu().numpy())[0])
            probabilities.append(confidence.item() * 100)
    
    return predictions, probabilities

# Test with a sample job description
sample_description = """
Leading and managing a team of pastoral care staff, providing guidance and support to students.
Developing and implementing pastoral care programs and initiatives.
Conducting one-on-one counseling sessions with students facing personal or academic challenges.
Coordinating with teachers and parents to ensure comprehensive support for students' wellbeing.
"""

predictions, confidences = predict_labels(sample_description)

print("\
Predictions for Sample Job Description:")
print("Question 40:", predictions[0], f"(Confidence: {confidences[0]:.2f}%)")
print("Question 41:", predictions[1], f"(Confidence: {confidences[1]:.2f}%)")
print("Question 42:", predictions[2], f"(Confidence: {confidences[2]:.2f}%)")

Model Accuracy on Test Set: 74.71%
Predictions for Sample Job Description:
Question 40: Response A (Confidence: 77.79%)
Question 41: Response A (Confidence: 78.85%)
Question 42: Response A (Confidence: 76.85%)


In [15]:
# Function to predict labels for a new job description without percentages
def predict_labels_simple(job_description):
    model.eval()
    encoding = tokenizer(
        job_description,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=512
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predictions = []
        
        for i in range(3):  # For each question
            _, preds = torch.max(outputs[:, i], dim=1)
            predictions.append(le.inverse_transform(preds.cpu().numpy())[0])
    
    return predictions

# Test with a sample job description
sample_description = """
The Web Services Manager, part of the Marketing, Communications, and Recruitment Department (Grade 8a), works 35 hours per week, typically from 9:00 am to 5:00 pm, Monday to Friday, with a one-hour lunch break. They are responsible to the Head of Marketing and Recruitment and oversee the Web Developer. The role involves managing the technical infrastructure and framework for Web Services, including the externally facing website and the Web Content Management System (WCMS), to ensure a seamless user experience that aligns with institutional objectives. Key duties include managing the institutional CMS, developing responsive style templates, integrating the CMS with corporate systems, and overseeing the deployment of bespoke code like HTML, CSS, and JavaScript for high-performing, SEO-optimized, user-friendly, and secure websites. Additionally, the manager leads the development of the digital governance framework, including maintaining the University's Pattern Library, and oversees the development of web tools that enhance user experience. They act as the main point of contact for web-related technologies, plan future web services developments, generate reports for leadership, and ensure the effective implementation of the University’s Web Strategy. Other responsibilities include user testing, using analytics to enhance functionality, managing the web budget, documenting code and processes, collaborating with Marketing, Recruitment, and Communications team members, liaising with external suppliers, and ensuring the upkeep of associated systems like the Search Tool and Asset Management Library. The Web Services Manager also supports digital transformation by promoting new technologies and trends, manages web support duties with Digital Team members, ensures secure system hosting in collaboration with IT services, and trains staff on CMS usage. Leadership responsibilities include managing the Web Developer, fostering teamwork, and driving transformational digital change. The role also involves compliance with GDPR, health and safety regulations, and the University's Equality, Diversity, and Inclusion Policy, with flexibility to undertake additional duties as required. The job description is subject to change to accommodate institutional developments, last updated in July 2021
"""

predictions = predict_labels_simple(sample_description)

print("Predictions for Sample Job Description:")
print("Question 40:", predictions[0])
print("Question 41:", predictions[1])
print("Question 42:", predictions[2])

Predictions for Sample Job Description:
Question 40: Response B
Question 41: Response B
Question 42: Response B


In [16]:
# Function to predict labels for a new job description without percentages
def predict_labels_simple(job_description):
    model.eval()
    encoding = tokenizer(
        job_description,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=512
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predictions = []
        
        for i in range(3):  # For each question
            _, preds = torch.max(outputs[:, i], dim=1)
            predictions.append(le.inverse_transform(preds.cpu().numpy())[0])
    
    return predictions

# Test with a sample job description
sample_description = """
The purpose of the role is to contribute to the delivery of taught modules and extra-curricular sessions for undergraduate, postgraduate, apprenticeship, and work-based programmes by providing lived experiences as a guest speaker, aiding curriculum development, and advising on opportunities for the University in West Yorkshire. Reporting to the Director of the Centre for Apprenticeships, Work-Based Learning, and Skills, the key responsibilities include developing and delivering workshops on important topics relevant to the West Yorkshire region, such as race, ethnicity, religion, and economic and social history. Additionally, the role involves contributing to extra-curricular activities and Equality & Diversity events hosted by the University or its partners, supporting recruitment activities, and maintaining links with external organizations to support these contributions. The individual will also advise on expanding the Ambassador network, identifying gaps in provision, and enhancing the student experience through further expertise. Furthermore, they will set up additional activities to raise the University's external profile. General duties include ensuring data use complies with regulations, particularly GDPR, adhering to health, safety, and wellbeing policies, promoting safeguarding and protection of others, applying the University's Equality, Diversity, and Inclusion Policy, and performing other duties as directed by their line manager. This job description is subject to variation by the Vice-Chancellor to reflect institutional developments
"""

predictions = predict_labels_simple(sample_description)

print("Predictions for Sample Job Description:")
print("Question 40:", predictions[0])
print("Question 41:", predictions[1])
print("Question 42:", predictions[2])

Predictions for Sample Job Description:
Question 40: Response A
Question 41: Response A
Question 42: Response A


In [17]:
# Function to predict labels for a new job description without percentages
def predict_labels_simple(job_description):
    model.eval()
    encoding = tokenizer(
        job_description,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=512
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predictions = []
        
        for i in range(3):  # For each question
            _, preds = torch.max(outputs[:, i], dim=1)
            predictions.append(le.inverse_transform(preds.cpu().numpy())[0])
    
    return predictions

# Test with a sample job description
sample_description = """
The purpose of the role is to create, implement, and manage sustainable solutions for the University's needs across all campus locations by evaluating the environmental impact of day-to-day activities and developments. The Sustainability Manager is responsible for supporting the management, monitoring, and reporting of the institution’s efforts to reduce its carbon footprint across scopes 1, 2, and 3 emissions, managing waste, recycling, and construction activities, and sourcing sustainable energy. The role also includes implementing new green technologies to achieve a net zero carbon status and minimizing the environmental impact of daily operations. Reporting to the Estates Project Development Manager, key responsibilities include sourcing solutions to reduce the University's environmental impact in areas such as materials, waste, energy, and water management, and ensuring compliance with environmental legislation. The role involves conducting ecology surveys, performing environmental impact assessments, and completing mandatory assessments like BREEAM. The manager will collect and analyze data to produce reports on the University's current status and recommend strategies for improvement, including cost-benefit analyses and timelines for implementation. They will advise on compliance with environmental law and suggest cost-effective technologies to lower energy emissions and waste. The role requires staying updated on research and legislation through seminars, publications, and social media. The Sustainability Manager will manage sustainability projects, prepare project bid documents, and ensure compliance with University policies. They will also oversee sustainability initiatives, ensuring the effective use of resources, compliance with legal requirements for waste management, and providing sustainability guidance on construction projects. The role includes maintaining the University’s sustainability and environmental policies and contributing to the management of carbon footprint reduction. The manager will work closely with the Head of Estates and Facilities and the Head of Commercial Enterprises to ensure sustainable and environmental responsibilities are managed effectively. General duties include ensuring compliance with GDPR, adhering to health, safety, and wellbeing policies, promoting safeguarding, applying the University’s Equality, Diversity, and Inclusion Policy, and performing other duties as directed by the line manager. This job description is subject to variation by the Vice-Chancellor to reflect institutional developments.
"""

predictions = predict_labels_simple(sample_description)

print("Predictions for Sample Job Description:")
print("Question 40:", predictions[0])
print("Question 41:", predictions[1])
print("Question 42:", predictions[2])

Predictions for Sample Job Description:
Question 40: Response A
Question 41: Response A
Question 42: Response A


In [18]:
# Function to predict labels for a new job description without percentages
def predict_labels_simple(job_description):
    model.eval()
    encoding = tokenizer(
        job_description,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=512
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predictions = []
        
        for i in range(3):  # For each question
            _, preds = torch.max(outputs[:, i], dim=1)
            predictions.append(le.inverse_transform(preds.cpu().numpy())[0])
    
    return predictions

# Test with a sample job description
sample_description = """
The Web Services Manager, part of the Marketing, Communications, and Recruitment Department (Grade 8a), works 35 hours per week, typically from 9:00 am to 5:00 pm, Monday to Friday, with a one-hour lunch break. They are responsible to the Head of Marketing and Recruitment and oversee the Web Developer. The role involves managing the technical infrastructure and framework for Web Services, including the externally facing website and the Web Content Management System (WCMS), to ensure a seamless user experience that aligns with institutional objectives. Key duties include managing the institutional CMS, developing responsive style templates, integrating the CMS with corporate systems, and overseeing the deployment of bespoke code like HTML, CSS, and JavaScript for high-performing, SEO-optimized, user-friendly, and secure websites. Additionally, the manager leads the development of the digital governance framework, including maintaining the University's Pattern Library, and oversees the development of web tools that enhance user experience. They act as the main point of contact for web-related technologies, plan future web services developments, generate reports for leadership, and ensure the effective implementation of the University’s Web Strategy. Other responsibilities include user testing, using analytics to enhance functionality, managing the web budget, documenting code and processes, collaborating with Marketing, Recruitment, and Communications team members, liaising with external suppliers, and ensuring the upkeep of associated systems like the Search Tool and Asset Management Library. The Web Services Manager also supports digital transformation by promoting new technologies and trends, manages web support duties with Digital Team members, ensures secure system hosting in collaboration with IT services, and trains staff on CMS usage. Leadership responsibilities include managing the Web Developer, fostering teamwork, and driving transformational digital change. The role also involves compliance with GDPR, health and safety regulations, and the University's Equality, Diversity, and Inclusion Policy, with flexibility to undertake additional duties as required. The job description is subject to change to accommodate institutional developments, last updated in July 2021
"""

predictions = predict_labels_simple(sample_description)

print("Predictions for Sample Job Description:")
print("Question 40:", predictions[0])
print("Question 41:", predictions[1])
print("Question 42:", predictions[2])

Predictions for Sample Job Description:
Question 40: Response B
Question 41: Response B
Question 42: Response B
